In [2]:
import pandas as pd
import numpy as np
import pathlib
import datetime
import xlrd
import openpyxl
import configparser
import os
import copy

In [42]:
#Проверка на одном здании
# Read config
config = configparser.ConfigParser()
config.read('configurations.ini')
config.sections()
#path_proj = config['path']['orc']
path_proj = config['path']['orc']
path_building = '27 Гараж_7 этап'

list_folder = os.listdir(path_proj + r'/' + path_building)
f = filter(lambda x: 'xlsx' in x, list_folder)
list_folder = list(f)
list_folder
for i in range (len(list_folder)):
    if (r'~$' in list_folder[i]):
        list_folder.pop(i)
df_folder = pd.Series(list_folder)
df_folder = df_folder.str.split('.xlsx', expand=True)
df_folder = df_folder.drop(1, axis = 1)
df_folder
df_folder_codes = df_folder[0].str.split('_', expand=True)
df_folder_codes.rename(columns= {0: 'project', 
1: 'stage',
2: 'phase',
3: 'building',
4: 'company',
5: 'type_file',
6: 'discipline',
7: 'discipline_rus',
8: 'description'},
inplace=True)

xls_list = list(list_folder)
listsht = list(list_folder)
listdf = list(list_folder)
df_mc = list(list_folder)
df_mcmt = list(list_folder)
df_m = list(list_folder)
df_room = list(list_folder)

for i in range(len(list_folder)):
    xls_list[i] = openpyxl.load_workbook(path_proj + r'/' + path_building + r'/' + list_folder[i])
    listsht[i] = xls_list[i].sheetnames
    listdf[i] = list(listsht[i])
    
#Заполнение списка датафремов 
    for j in range(len(listsht[i])): 
        listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
        if (listdf[i][j].size == 0):
            listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count'])  
#Заполнение пустых спецификаций
    for j in range(len(listsht[i])): 
        listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
        if (listdf[i][j].size == 0 or 'Department' in listdf[i][j].columns):
            listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count'])  
            
#Установка мульти категорий           
    df_mc[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Multi-Category')
    df_mcmt[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Multi-Category MT')
#Объединение мульти и мульти материал
    df_m[i] = pd.concat([df_mcmt[i],df_mc[i]], join = "outer")  
#Установка датафр помещений  
    if 'VTBR_Room' in listsht[i]:
        df_room[i] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = 'VTBR_Room') 
#Удаление мульти спец 
    for j in range(len(listsht[i])):         
        if ('Category' in listdf[i][j].columns):
            listdf[i].append(listdf[i][j])
            del listdf[i][j]
    for j in range(len(listsht[i])):           
        if ('Category' in listdf[i][j].columns):
            listdf[i].append(listdf[i][j])
            del listdf[i][j]
    del listdf[i][-2:]    
#Добавление Category 
for i in range(len(listdf)):
    for j in range(len(listdf[i])):          
            listdf[i][j].insert(0, 'Building', df_folder_codes.building[i])
            listdf[i][j].insert(1, 'Discipline', df_folder_codes.discipline[i])
            listdf[i][j].insert(2, 'Category', (listdf[i][j].merge(df_m[i], on = 'Family and Type', how = 'left').Category))
            listdf[i][j].insert(3, 'File', list_folder[i])            
#Объединение спецификаций 
df_cnc = list(listdf)
for i in range(len(listdf)):
    df_cnc[i] = pd.concat([listdf[i][0]
                           ], join = "outer")
    for j in range(1, len(listdf[i])):
        df_cnc[i] = pd.concat([df_cnc[i],
                            listdf[i][j]
                           ], join = "outer")

df_cnc_building = pd.concat([df_cnc[0]
                           ], join = "outer")
for i in range(1, len(df_cnc)):
    df_cnc_building = pd.concat([df_cnc_building,
                            df_cnc[i]
                           ], join = "outer")
    
            

AttributeError: 'DataFrame' object has no attribute 'Category'

In [52]:
listsht[9]


['VTBR_Cable Tray',
 'VTBR_Cable Tray Fitting',
 'VTBR_Casework',
 'VTBR_Ceiling',
 'VTBR_Column',
 'VTBR_Communication Device',
 'VTBR_Curtain Panel',
 'VTBR_Data Device',
 'VTBR_Door',
 'VTBR_Duct',
 'VTBR_Duct Accessory',
 'VTBR_Duct Fitting',
 'VTBR_Duct Insulation',
 'VTBR_Electrical Equipment',
 'VTBR_Electrical Fixture',
 'VTBR_Fire Alarm Device',
 'VTBR_Flex Duct',
 'VTBR_Flex Pipe',
 'VTBR_Floor',
 'VTBR_Furniture',
 'VTBR_Furniture System',
 'VTBR_Generic Model',
 'VTBR_Lighting Device',
 'VTBR_Lighting Fixture',
 'VTBR_Mechanical Equipment',
 'VTBR_Multi-Category',
 'VTBR_Multi-Category MT',
 'VTBR_Multi-Category MT1',
 'VTBR_Multi-Category1',
 'VTBR_Nurse Call Device',
 'VTBR_Pipe',
 'VTBR_Pipe Accessory',
 'VTBR_Pipe Fitting',
 'VTBR_Pipe Insulation',
 'VTBR_Plumbing Fixture',
 'VTBR_Roof',
 'VTBR_Room',
 'VTBR_Security Device']

In [39]:
listdf[12][19]
    

,Family and Type,О_Наименование,О_Завод-изготовитель,О_Единица измерения,5D_ВТБн_Код классификатора,5D_ВТБн_Описание кода классификатора,ВТБн_Дубликат,О_Обозначение,Count,О_Код изделия,Type Mark,Type Comments,Assembly Code,Assembly Description
0,СП_СОУЭ_Оповещатель рупорный: Рупор,Оповещатель звуковой 24 В 105дБ,"ООО """"Электротехника и автоматика",шт.,02.01.06.02.02.01.01,Оконечные устройства,NaN,МАЯК-24-ЗМ1,1,NaN,NaN,NaN,NaN,NaN
1,СП_СОУЭ_Оповещатель световой табло: Автоматика...,"Оповещатель световой """"Автоматика отключена!","ООО """"Системсервис",шт.,02.01.06.02.02.01.01,Оконечные устройства,NaN,КОП-25П IP54,1,NaN,NaN,NaN,NaN,NaN
2,СП_СОУЭ_Оповещатель световой табло: Газ Уходи,"Оповещатель световой """"Газ! Уходи!","ООО """"Системсервис",шт.,02.01.06.02.02.01.01,Оконечные устройства,NaN,КОП-25П IP54,1,NaN,NaN,NaN,NaN,NaN
3,СП_СОУЭ_Оповещатель световой табло: Газ не вхо...,"Оповещатель световой """"Газ! не входи!","ООО """"Системсервис",шт.,02.01.06.02.02.01.01,Оконечные устройства,NaN,КОП-25П IP54,1,NaN,NaN,NaN,NaN,NaN


In [4]:
true_lst = copy.deepcopy(listdf)
for i in range(len(listdf)):
    for j in range(len(listdf[i])):
        true_lst[i][j] = ('Family and Type' in listdf[i][j].columns)
check_list = list(true_lst)
for i in range(len(listdf)):
    check_list[i] = all(true_lst[i])    
        

check_list       

[True, True, True, True, True, True, True, True, True, True, True, True, True]

In [35]:
listdf


[[  Building Discipline  Category  \
  0      1.3         AR  Ceilings   
  
                                                  File  \
  0  ORC_T_III_1.3_STH_BIM_AR_Архитект(ВГЗП)_Schedu...   
  
                     Family and Type  \
  0  Compound Ceiling: СП_АР_ГКЛВ_50   
  
                                        О_Наименование О_Завод-изготовитель  \
  0  1.Подвесной потолок ГВЛ  2.Краска водоэмульсио...     Knauf, Tikkurila   
  
    О_Единица измерения 5D_ВТБн_Код классификатора  \
  0                  м2       02.01.06.01.10.01.01   
  
    5D_ВТБн_Описание кода классификатора  ВТБн_Дубликат О_Обозначение  \
  0                     Отделка потолков            NaN             -   
  
     О_Позиция  Area  Count  Type Mark  Type Comments  Mark  
  0        NaN  3.35      1        NaN            NaN   NaN  ,
    Building Discipline Category  \
  0      1.3         AR    Doors   
  1      1.3         AR    Doors   
  2      1.3         AR    Doors   
  3      1.3         AR    Door

In [9]:
for i in range(len(listdf)):
    print(all(true_lst[i]))
        

True
True
True
True
True
True
True
False
True
True


In [78]:
#Проверка на одном здании
# Read config
config = configparser.ConfigParser()
config.read('configurations.ini')
config.sections()
#path_proj = config['path']['orc']
path_proj = config['path']['tlc']
path_building = '5.3_КПП3'

list_folder = os.listdir(path_proj + r'/' + path_building)
f = filter(lambda x: 'xlsx' in x, list_folder)
list_folder = list(f)
list_folder
for i in range (len(list_folder)):
    if (r'~$' in list_folder[i]):
        list_folder.pop(i)
df_folder = pd.Series(list_folder)
df_folder = df_folder.str.split('.xlsx', expand=True)
df_folder = df_folder.drop(1, axis = 1)
df_folder
df_folder_codes = df_folder[0].str.split('_', expand=True)
df_folder_codes.rename(columns= {0: 'project', 
1: 'stage',
2: 'phase',
3: 'building',
4: 'company',
5: 'type_file',
6: 'discipline',
7: 'discipline_rus',
8: 'description'},
inplace=True)

xls_list = list(list_folder)
listsht = list(list_folder)
listdf = list(list_folder)
df_mc = list(list_folder)
df_mcmt = list(list_folder)
df_m = list(list_folder)
df_room = list(list_folder)
for i in range(len(list_folder)):
    xls_list[i] = openpyxl.load_workbook(path_proj + r'/' + path_building + r'/' + list_folder[i])
    listsht[i] = xls_list[i].sheetnames
    listdf[i] = list(listsht[i])
#Заполнение списка датафремов 
    for j in range(len(listsht[i])): 
        listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
        if (listdf[i][j].size == 0):
            listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count'])  
#Заполнение пустых спецификаций
    for j in range(len(listsht[i])): 
        listdf[i][j] = pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[i], sheet_name = listsht[i][j])
        if (listdf[i][j].size == 0 or 'Department' in listdf[i][j].columns):
            listdf[i][j] = pd.DataFrame(columns=['Family and Type', 'Count']) 


In [79]:
list_folder

['TLC_P_II_5.3_MK3_BIM_AR_Архитектура_Schedules.xlsx',
 'TLC_P_II_5.3_MK3_BIM_ES_Электроснабжение_Schedules.xlsx',
 'TLC_P_II_5.3_MK3_BIM_KR_Конструктив_Schedules.xlsx',
 'TLC_P_II_5.3_MK3_BIM_OV_ОВиК_Schedules.xlsx',
 'TLC_P_II_5.3_MK3_BIM_PS_Пожарная сигнализация_Schedules.xlsx',
 'TLC_P_II_5.3_MK3_BIM_SS_Слаботочные системы_Schedules.xlsx',
 'TLC_P_II_5.3_MK3_BIM_TX_Технология_Schedules.xlsx',
 'TLC_P_II_5.3_MK3_BIM_VK_Водоснабжение и канализация_Schedules.xlsx']

In [80]:
xls_list

In [85]:
listsht[3]

['VTBR_Mechanical Equipment',
 'VTBR_Duct Fitting',
 'VTBR_Duct Accessory',
 'VTBR_Air Terminal',
 'VTBR_Duct',
 'VTBR_Pipe Accessory',
 'VTBR_Pipe Fitting',
 'VTBR_Pipe',
 'VTBR_Duct Insulation',
 'VTBR_Pipe Insulation',
 'VTBR_Flex Duct']

In [74]:
pd.read_excel(path_proj + r'/' + path_building + r'/' + list_folder[3], sheet_name = 'VTBR_Multi-Category')

ValueError: Worksheet named 'VTBR_Multi-Category' not found

In [69]:
# Проверка Family and Type
list_true = list(listdf)
for i in range(len(listdf)):
    print(i)
    for j in range(len(listdf[i])):
        list_true[i].append
        print('Family and Type' in listdf[i][j],'---', j)


0
False --- 0
False --- 1
False --- 2
False --- 3
False --- 4
False --- 5
False --- 6
False --- 7
False --- 8
False --- 9
False --- 10
False --- 11
False --- 12
1
False --- 0
False --- 1
False --- 2
False --- 3
False --- 4
False --- 5
False --- 6
False --- 7
False --- 8
False --- 9
False --- 10
2
False --- 0
False --- 1
False --- 2
False --- 3
False --- 4
False --- 5
False --- 6
False --- 7
False --- 8
False --- 9
False --- 10
False --- 11
False --- 12
False --- 13
False --- 14
False --- 15
False --- 16
False --- 17
False --- 18
False --- 19
False --- 20
False --- 21
False --- 22
False --- 23
False --- 24
False --- 25
False --- 26
False --- 27
False --- 28
False --- 29
False --- 30
False --- 31
False --- 32
False --- 33
False --- 34
False --- 35
False --- 36
False --- 37
False --- 38
False --- 39
False --- 40
False --- 41
False --- 42
False --- 43
False --- 44
False --- 45
False --- 46
False --- 47
False --- 48
3
False --- 0
False --- 1
False --- 2
False --- 3
False --- 4
False --- 5
F

In [45]:
# Проверка мульти
list_true_mt = list(list_folder)
for i in range(len(list_folder)):
    list_true_mt[i] = 'VTBR_Multi-Category MT' in listsht[i]


list_true_mt

        

[True, True, True, True, True, True, True, True, True, True]

In [42]:
listsht[1]

['VTBR_Cable Tray',
 'VTBR_Cable Tray Fitting',
 'VTBR_Electrical Equipment',
 'VTBR_Lighting Fixture',
 'VTBR_Multi-Category',
 'VTBR_Multi-Category MT',
 'VTBR_Security Device']

In [43]:
#Проверка Multi
for i in range(len(listsht)):
    print('VTBR_Multi-Category MT' in listsht[i])

True
True
True
True
True
True
True
True
True
True


In [87]:
listsht[0]

['VTBR_Multi-Category MT',
 'VTBR_Floor',
 'VTBR_Structural Column',
 'VTBR_Structural Foundation',
 'VTBR_Structural Framing',
 'VTBR_Wall',
 'VTBR_Panel',
 'VTBR_Door',
 'VTBR_Window',
 'VTBR_Plumbing Fixture',
 'VTBR_Multi-Category',
 'VTBR_Structural Connection',
 'VTBR_Generic Model']